In [1]:
import pandas as pd
import regex as re
import numpy as np
import urllib3
import requests
import sqlite3
from IPython.display import display, clear_output
import json
from datetime import date,datetime
today = date.today().strftime("%Y-%m-%d")
base = 'https://statsapi.web.nhl.com'

In [2]:
def convertDates(date_val):
    """
    convertCheckDates:
    Description: Checks for actual dates and returns date 'YYYY-MM-DD'.
    If not an actual date will fail and flag an assertion error.
    """
    # accepted date values to convert to string dates
    acc_values = (type(datetime.today()), type(date.today()))
    
    # if acceptable date than return as string date
    if type(date_val) in  acc_values: 
        return date_val.date()
   
    # regular expression to validate string dates
    p = re.compile('\d{4}-\d{1,2}-\d{1,2}')
    m = p.match(date_val)
   
    # if not a correct date than will flag an error
    assert(m), """Date was entered incorrectly should be a date field or string date 'YYYY-MM-DD'"""
    
    return datetime.strptime(date_val, '%Y-%m-%d').date()


In [33]:
def getNHLGames(start = today, end = today):
    """
    getNHLGames:
    Description: This will pull all NHL games based on the variables pased through. If variables are not 
    passed will defualt to the current date.
    """
    start = convertDates(start)
    end  = convertDates(end)
    # flag if start date is greater than end 
    assert(start <= end), "Start date ({}) needs to be less than the end date ({})".format(start, end)
    
    url = base + '/api/v1/schedule?&startDate={}&endDate={}'.format(start,end)
    try: 
        resp = requests.get(url)

        # just making sure we have a response
        assert(resp.status_code == 200), '''{}: Nothing returned check the link {}'''.format(resp.status_code, url)
   
    except urllib3.exceptions.HTTPError as e:
        print('Well, we tried but, {}'.format(e))
    
    return resp

In [35]:
def setGamesToDF(data):
    """
    setGamesToDF: 
    Description: Takes the response for the getNHLGames call and converts the data to a pandas data frame.
    """
    games= {}
    for i in data['dates']:
        for ii in i['games']:
            games[ii['gamePk']] = {'link' : ii['link'], 
                          'season' : ii['season'], 
                          'gameDate' : ii['gameDate'], 
                          'gameType': str(ii['gamePk'])[4:6],
                          'awayTeamId' : ii['teams']['away']['team']['id'], 
                          'homeTeamId' : ii['teams']['home']['team']['id']
                         }
    df = pd.DataFrame.from_dict(games, orient = 'index')
    return df 

In [36]:
resp = getNHLGames('2014-10-01')
nhlGames = setGamesToDF(resp.json())
nhlGames.head()

,link,season,gameDate,gameType,awayTeamId,homeTeamId
2014010077,/api/v1/game/2014010077/feed/live,20142015,2014-10-01T23:00:00Z,01,15,7
2014010079,/api/v1/game/2014010079/feed/live,20142015,2014-10-01T23:00:00Z,01,12,29
2014010081,/api/v1/game/2014010081/feed/live,20142015,2014-10-01T23:30:00Z,01,5,17
2014010078,/api/v1/game/2014010078/feed/live,20142015,2014-10-02T00:30:00Z,01,8,16
2014010080,/api/v1/game/2014010080/feed/live,20142015,2014-10-02T01:00:00Z,01,53,22


In [38]:
def pullGameData(link): 
    """
    pullGameData:
    Description: Pulls individual game data from the nhl website and returns the repsonse.
    """
    # regular expression to validate the link
    p = re.compile('/api/v1/game/\d{10}/feed/live')
    m = p.search(link)
    assert(m), """{} passed to the function 'pullGameData' should be '/api/v1/game/[gameid]/feed/live'""".format(link)
    
    # in case the full path is passed through, drop base and add back in
    link = m[0]
    url = base + link
    
    try:
        resp = requests.get(url)
        
        # just making sure we have a response
        assert(resp.status_code == 200), '''{}: Nothing returned check the link {}'''.format(resp.status_code, link)
    
    except urllib3.exceptions.HTTPError as e:
        print('Well, we tried but, {}'.format(e))
    
    return resp

In [59]:
def gameEventsDictionary(data): 
    """
    gameEventsDictionary:
    Description: Game data is passed through as json for each game, and this function pulls all the events
    from the game passed and passes it back as a dictionary.
    """
    #gameid
    gamePk = str(data['gamePk'])
    gameDict = {}
    
    # liveData is where all the game information is stored
    for gEvent in data['liveData']['plays']['allPlays']:
        
        try:
            _dict_ = {}
            _dict_['event'] = gEvent['result']['event']
            _dict_['period'] = gEvent['about']['period']
            _dict_['periodType'] = gEvent['about']['periodType']
            _dict_['periodTime'] = gEvent['about']['periodTime']
            _dict_['periodTimeRemaining'] : gEvent['about']['periodTimeRemaining']
            _dict_['dateTime'] = gEvent['about']['dateTime']
            
            # tracking the points on each event
            _dict_['home'] = gEvent['about']['goals']['home']
            _dict_['away'] = gEvent['about']['goals']['away']
            _dict_['description'] = gEvent['result']['description']
            
            # x and y coordinates are stored here
            if len(gEvent['coordinates']) > 0:
                # ran into instances where just one coordinate was populated
                for k,v in gEvent['coordinates'].items():
                    _dict_[k] = v
                    
            if 'team' in gEvent.keys():
                _dict_['eventTeam'] = gEvent['team']['id']
            
            if 'players' in gEvent.keys():
                for i in range(len(gEvent['players'])):
                    _dict_['playerid' + str(i+1)] = gEvent['players'][i]['player']['id']
                    _dict_['playerType' + str(i+1)] = gEvent['players'][i]['playerType']

        except:
            # trying to give enough information to know which game event failed
            print('{} - failed at line {}'.format(gamePk, gEvent['about']['eventIdx']))
            
        gameDict[int(gamePk + str( gEvent['about']['eventIdx']))] =_dict_
    return gameDict
    

In [64]:
def getEvents(data):
    """
    getEvents:
    This is where all the data is pulled together. The function calls pullGameData, and appends a dictionary
    for each game that was played.
    """
    _dict_={}
    n = len(data) - 1
    ii = 0
    for i in data:
        # update bar used to clear out output and refresh progress
        clear_output(wait=True) 
        
        # calculating percent completed
        print('{} - {} % complete'.format(ii, ii/n * 100))
        
        # calls data based on link for game
        _resp_ = pullGameData(i)
        
        # gameEventsDictionary breaks each game event out line by line
        _dict_.update(gameEventsDictionary(_resp_.json()))
        
        ii += 1 # increment counter for % completed
        
    # create dictionary to return the data
    df = pd.DataFrame.from_dict(_dict_, orient = 'index')
    
    return df

In [65]:
eventTable = getEvents(nhlGames.link.values)

8330 - 100.0 % complete


In [66]:
eventTable.head()

,event,period,periodType,periodTime,dateTime,home,away,description,x,y,eventTeam,playerid1,playerType1,playerid2,playerType2,playerid3,playerType3,playerid4,playerType4
20140100770,Game Scheduled,1,REGULAR,00:00,2014-10-01T21:15:40Z,0,0,Game Scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20140100771,Period Ready,1,REGULAR,00:00,2014-10-01T22:53:59Z,0,0,Period Ready,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20140100772,Period Start,1,REGULAR,00:00,2014-10-01T23:07:19Z,0,0,Period Start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20140100773,Faceoff,1,REGULAR,00:00,2014-10-01T23:07:19Z,0,0,Tyler Ennis faceoff won against Andre Burakovsky,0.0,0.0,7.0,8474589.0,Winner,8477444.0,Loser,NaN,NaN,NaN,NaN
20140100774,Hit,1,REGULAR,00:15,2014-10-01T23:08:22Z,0,0,Brooks Laich hit Tyler Myers,-74.0,-40.0,15.0,8469639.0,Hitter,8474574.0,Hittee,NaN,NaN,NaN,NaN


In [67]:
eventTable.res

Index(['event', 'period', 'periodType', 'periodTime', 'dateTime', 'home',
       'away', 'description', 'x', 'y', 'eventTeam', 'playerid1',
       'playerType1', 'playerid2', 'playerType2', 'playerid3', 'playerType3',
       'playerid4', 'playerType4'],
      dtype='object')

In [76]:
# storing data in sql
query = """ 
            CREATE TABLE DIM_EVENT_TABLE
            (   id INTEGER,
                event VARCHAR(50),
                period INTEGER,
                periodType VARCHAR(50),
                periodTime VARCHAR(10),
                dateTime VARCHAR(30),
                home INTEGER,
                away INTEGER,
                description VARCHAR(200),
                x INTEGER,
                y INTEGER,
                eventTeam INTEGER,
                playerid1 INTEGER,
                playerType1 VARCHAR(50),
                playerid2 INTEGER,
                playerType2 VARCHAR(50),
                playerid3 INTEGER,
                playerType3 VARCHAR(50),
                playerid4 INTEGER,
                playerType4 VARCHAR(50)
            );"""

In [77]:
# now we need to create our database or place to store the table
con = sqlite3.connect('dsc_550_project.sqlite') 
# commit, keeps the entire transaction, else it will be rolled back once we log out
con.execute(query)
# excecute the create table statement so we have a table set up
con.commit()

In [81]:
#insert statement, '?' are like the placeholders for the variables
stmt = "INSERT INTO DIM_EVENT_TABLE VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
# normally in sql you would have the insert statemnt above and then the data below
# doing it like this tells sqlite to insert and then we have the values after the comma
con.executemany(stmt, eventTable.reset_index().values.tolist())
con.commit()

In [87]:
# creating sel * to see the data
cursor = con.execute('Select * from DIM_EVENT_TABLE')
rows = cursor.fetchall()
rows

[(20140100770,
  'Game Scheduled',
  1,
  'REGULAR',
  '00:00',
  '2014-10-01T21:15:40Z',
  0,
  0,
  'Game Scheduled',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (20140100771,
  'Period Ready',
  1,
  'REGULAR',
  '00:00',
  '2014-10-01T22:53:59Z',
  0,
  0,
  'Period Ready',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (20140100772,
  'Period Start',
  1,
  'REGULAR',
  '00:00',
  '2014-10-01T23:07:19Z',
  0,
  0,
  'Period Start',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (20140100773,
  'Faceoff',
  1,
  'REGULAR',
  '00:00',
  '2014-10-01T23:07:19Z',
  0,
  0,
  'Tyler Ennis faceoff won against Andre Burakovsky',
  0,
  0,
  7,
  8474589,
  'Winner',
  8477444,
  'Loser',
  None,
  None,
  None,
  None),
 (20140100774,
  'Hit',
  1,
  'REGULAR',
  '00:15',
  '2014-10-01T23:08:22Z',
  0,
  0,
  'Brooks Laich hit Tyler Myers',
  -74,
  -40,
  15

In [88]:
# storing data in sql
query = """ 
            CREATE TABLE DIM_SEASON_TABLE
            ( 
                id INTEGER,
                link VARCHAR(100),
                season INTEGER,
                gameDate VARCHAR(50),
                gameType VARCHAR(50),
                awayTeamId INTEGER,
                homeTeamId INTEGER
            );"""

In [89]:
# now we need to create our database or place to store the table
con = sqlite3.connect('dsc_550_project.sqlite') 
# commit, keeps the entire transaction, else it will be rolled back once we log out
con.execute(query)
# excecute the create table statement so we have a table set up
con.commit()

In [90]:
#insert statement, '?' are like the placeholders for the variables
stmt = "INSERT INTO DIM_SEASON_TABLE VALUES(?,?,?,?,?,?,?)"
# normally in sql you would have the insert statemnt above and then the data below
# doing it like this tells sqlite to insert and then we have the values after the comma
con.executemany(stmt, nhlGames.reset_index().values.tolist())
con.commit()

In [91]:
# creating sel * to see the data
cursor = con.execute('Select * from DIM_SEASON_TABLE')
rows = cursor.fetchall()
rows

[(2014010077,
  '/api/v1/game/2014010077/feed/live',
  20142015,
  '2014-10-01T23:00:00Z',
  '01',
  15,
  7),
 (2014010079,
  '/api/v1/game/2014010079/feed/live',
  20142015,
  '2014-10-01T23:00:00Z',
  '01',
  12,
  29),
 (2014010081,
  '/api/v1/game/2014010081/feed/live',
  20142015,
  '2014-10-01T23:30:00Z',
  '01',
  5,
  17),
 (2014010078,
  '/api/v1/game/2014010078/feed/live',
  20142015,
  '2014-10-02T00:30:00Z',
  '01',
  8,
  16),
 (2014010080,
  '/api/v1/game/2014010080/feed/live',
  20142015,
  '2014-10-02T01:00:00Z',
  '01',
  53,
  22),
 (2014010085,
  '/api/v1/game/2014010085/feed/live',
  20142015,
  '2014-10-02T23:00:00Z',
  '01',
  2,
  1),
 (2014010086,
  '/api/v1/game/2014010086/feed/live',
  20142015,
  '2014-10-02T23:00:00Z',
  '01',
  4,
  15),
 (2014010088,
  '/api/v1/game/2014010088/feed/live',
  20142015,
  '2014-10-02T23:30:00Z',
  '01',
  14,
  13),
 (2014010087,
  '/api/v1/game/2014010087/feed/live',
  20142015,
  '2014-10-03T00:00:00Z',
  '01',
  30,
  19)